In [1]:
import numpy as np

# house (1) (house3D_final_new2_noverhang)

In [ ]:
def house(self, o, eg_length, eg_width, eg_height, roof_length, roof_width, angle):
    o[2] = 0
    angle = angle * np.pi / 180
    inside_mask = np.zeros_like(self.grid.global_grid()[0], dtype=bool)
    inside_mask = np.where(np.logical_and(
        np.logical_and(np.logical_and(o[0] - eg_length / 2 < self.grid.global_grid()[0], self.grid.global_grid()[0] < o[0] + eg_length / 2),
                       np.logical_and(o[1] - eg_width / 2 < self.grid.global_grid()[1], self.grid.global_grid()[1] < o[1] + eg_width / 2)),
                        np.logical_and(o[2] - eg_height <= self.grid.global_grid()[2], self.grid.global_grid()[2] <= o[2] + eg_height)), True, inside_mask)
    inside_mask = np.where(np.logical_and(np.logical_and(
                    np.logical_and(np.logical_and(o[0] - roof_length / 2 < self.grid.global_grid()[0], self.grid.global_grid()[0] < o[0] + roof_length / 2),
                    np.logical_and(o[1] - roof_width / 2 < self.grid.global_grid()[1], self.grid.global_grid()[1] < o[1] + roof_width / 2)),
                    np.logical_and(o[2] + eg_height < self.grid.global_grid()[2],
                    self.grid.global_grid()[2] < o[2] + eg_height + 0.001 + np.tan(angle) * (self.grid.global_grid()[0] - o[0] + roof_width / 2))),
                    self.grid.global_grid()[2] < o[2] + eg_height + 0.001 - np.tan(angle) * (self.grid.global_grid()[0] - o[0] - roof_width / 2)), True, inside_mask)

    """make masks for fs to be bounced / not streamed by going over all obstacle points and
    following all e_i's to find neighboring points and which of their fs point towards the obstacle
    (fs pointing to obstacle are added to no_stream_mask, fs pointing away are added to bouncedFs)"""

    x, y, z = inside_mask.shape
    outgoing_mask = np.zeros((self.units.lattice.Q, x, y, z), dtype=bool)
    a, b, c = np.where(inside_mask)
    for p in range(0, len(a)):
        for i in range(0, self.units.lattice.Q):
            try:  # try in case the neighboring cell does not exist (an f pointing out of simulation domain)
                if not inside_mask[a[p] + self.units.lattice.stencil.e[i, 0], b[p] + self.units.lattice.stencil.e[i, 1], c[p] + self.units.lattice.stencil.e[i, 2]]:
                    outgoing_mask[i, a[p] + self.units.lattice.stencil.e[i, 0], b[p] + self.units.lattice.stencil.e[i, 1], c[p] + self.units.lattice.stencil.e[i, 2]] = 1
            except IndexError:
                pass  # just ignore this iteration since there is no neighbor there

    measurement_points = []
    measurement_points.append(o + np.asarray([roof_length * 0.25, -roof_width * 0.25, 0]))
    measurement_points.append(o + np.asarray([roof_length * 0.75, -roof_width * 0.25, 0]))
    measurement_points.append(o + np.asarray([roof_length * 0.25, roof_width * 0.25, 0]))
    measurement_points.append(o + np.asarray([roof_length * 0.75, roof_width * 0.25, 0]))
    measurement_points.append(o + np.asarray([roof_length * 0.5, 0, 0]))

    for point in measurement_points:
        point[2] = eg_height + 0.001 - np.tan(angle) * (point[0] - o[0] - roof_width / 2)
        for i in range(0, 3):
            point[i] = int(self.units.convert_length_to_lu(point[i]))
        point[2] += 2

    return inside_mask, measurement_points

## house (kommt so in die Arbeit, simulation_code.py (!))


In [ ]:
def house(self, o, eg_x_length, eg_y_length, roof_height, roof_overhang, angle=35):
    """Outputs mask for flow: house with square base
        and gable-roof parallel to y direction and overhang

        Inputs:
        eg_x_length: length of the house base in x-direction, in LU
        eg_y_length: length of the house base in y-direction, in LU
        roof_height: height of the top of the roof, in LU

    """
    angle = angle * np.pi / 180
    eg_height = int(round(roof_height - (eg_x_length / 2 + roof_overhang)
                          * np.tan(angle)))
    self.roof_height = roof_height
    inside_mask = np.zeros_like(self.grid.global_grid()[0], dtype=bool)
    inside_mask[int(o[0]-eg_x_length/2):int(o[0]+eg_x_length/2),
                int(o[1]-eg_y_length/2):int(o[1]+eg_y_length/2),
                :eg_height] = True
    inside_mask[int(o[0]-eg_x_length/2-roof_overhang):
                int(o[0]+eg_x_length/2+roof_overhang),
                int(o[1]-eg_y_length/2):
                int(o[1]+eg_y_length/2), eg_height] = True
    inside_mask[int(o[0] - eg_x_length / 2-roof_overhang):
                int(o[0] + eg_x_length / 2+roof_overhang),
                int(o[1] - eg_y_length / 2):
                int(o[1] + eg_y_length / 2), eg_height+1:] = \
        np.where(self.units.convert_length_to_lu(
            self.grid()[2][int(o[0] - eg_x_length / 2-roof_overhang):
                           int(o[0] + eg_x_length / 2+roof_overhang),
                           int(o[1] - eg_y_length / 2):
                           int(o[1] + eg_y_length / 2), eg_height+1:]) <
            o[2] + roof_height + 0.5 - np.tan(angle) *
                np.abs(self.units.convert_length_to_lu(
                        self.grid()[0][
                        int(o[0] - eg_x_length / 2-roof_overhang):
                        int(o[0] + eg_x_length / 2+roof_overhang),
                        int(o[1] - eg_y_length / 2):
                        int(o[1] + eg_y_length / 2),
                        eg_height+1:]) - o[0]),
            True,
            inside_mask[int(o[0] - eg_x_length / 2-roof_overhang):
                        int(o[0] + eg_x_length / 2+roof_overhang),
                        int(o[1] - eg_y_length / 2):
                        int(o[1] + eg_y_length / 2), eg_height+1:])
    return inside_mask

## house (final)
EINZIGE Änderung zu house1 (final_new2_noverhang) ist "self.eg_height = eg_height" / auch wenns nicht verwendet wird. In der Funktion selbst wird immer der Input-Parameter verwendet

ÄNDERUNGEN zu ksiA und sim_code der Arbeit:
- Input: andere Benennung on Länge, Breite, Höhe des Hauses
- Andere Definition von Länge Breite Höhe des Hauses
-   ksiA hat overhang-Parameter
-   ksiA berechnet eg_height aus roof_height
- o[2]=0 Der Ursprung in z wird überschrieben
- komplett andere Belegung der inside_mask
-   final: Kombination aus torch.where und np.logical_and
-   ksiA: Indizierung von/bis in mehreren Teilen
- final definiert outgoing_mask -> sieht so aus, wie die f-Finderei der HWBB/IBB, also sollten das alle Pops sein, die aus dem Haus raus zeigen
-   ABER: outgoing_mask wird gar nicht verwendet und ist kein Attribut
- final definiert measurement_points
-   definiert erstmal in PU die MEsspunkte in der XY-Ebene und z=0
-   überführt dann über eine for point in measurement_points Schleife die points in die Korrekte z-Höhe? (Höhe, Tangens etc.)
-   gibt m.p.s zusammen mit der inside_mask zurück


In [ ]:
def house(self, o, eg_length, eg_width, eg_height, roof_length, roof_width, angle):
    o[2] = 0
    angle = angle * np.pi / 180
    self.eg_height = eg_height   ### DAS HIER
    inside_mask = np.zeros_like(self.grid.global_grid()[0], dtype=bool)
    inside_mask = np.where(np.logical_and(
        np.logical_and(np.logical_and(o[0] - eg_length / 2 < self.grid.global_grid()[0], self.grid.global_grid()[0] < o[0] + eg_length / 2),
                       np.logical_and(o[1] - eg_width / 2 < self.grid.global_grid()[1], self.grid.global_grid()[1] < o[1] + eg_width / 2)),
                        np.logical_and(o[2] - eg_height <= self.grid.global_grid()[2], self.grid.global_grid()[2] <= o[2] + eg_height)), True, inside_mask)
    inside_mask = np.where(np.logical_and(np.logical_and(
                    np.logical_and(np.logical_and(o[0] - roof_length / 2 < self.grid.global_grid()[0], self.grid.global_grid()[0] < o[0] + roof_length / 2),
                    np.logical_and(o[1] - roof_width / 2 < self.grid.global_grid()[1], self.grid.global_grid()[1] < o[1] + roof_width / 2)),
                    np.logical_and(o[2] + eg_height < self.grid.global_grid()[2],
                    self.grid.global_grid()[2] < o[2] + eg_height + 0.001 + np.tan(angle) * (self.grid.global_grid()[0] - o[0] + roof_width / 2))),
                    self.grid.global_grid()[2] < o[2] + eg_height + 0.001 - np.tan(angle) * (self.grid.global_grid()[0] - o[0] - roof_width / 2)), True, inside_mask)

    """make masks for fs to be bounced / not streamed by going over all obstacle points and
    following all e_i's to find neighboring points and which of their fs point towards the obstacle
    (fs pointing to obstacle are added to no_stream_mask, fs pointing away are added to bouncedFs)"""

    x, y, z = inside_mask.shape
    outgoing_mask = np.zeros((self.units.lattice.Q, x, y, z), dtype=bool)
    a, b, c = np.where(inside_mask)
    for p in range(0, len(a)):
        for i in range(0, self.units.lattice.Q):
            try:  # try in case the neighboring cell does not exist (an f pointing out of simulation domain)
                if not inside_mask[a[p] + self.units.lattice.stencil.e[i, 0], b[p] + self.units.lattice.stencil.e[i, 1], c[p] + self.units.lattice.stencil.e[i, 2]]:
                    outgoing_mask[i, a[p] + self.units.lattice.stencil.e[i, 0], b[p] + self.units.lattice.stencil.e[i, 1], c[p] + self.units.lattice.stencil.e[i, 2]] = 1
            except IndexError:
                pass  # just ignore this iteration since there is no neighbor there

    measurement_points = []
    measurement_points.append(o + np.asarray([roof_length * 0.25, -roof_width * 0.25, 0]))
    measurement_points.append(o + np.asarray([roof_length * 0.75, -roof_width * 0.25, 0]))
    measurement_points.append(o + np.asarray([roof_length * 0.25, roof_width * 0.25, 0]))
    measurement_points.append(o + np.asarray([roof_length * 0.75, roof_width * 0.25, 0]))
    measurement_points.append(o + np.asarray([roof_length * 0.5, 0, 0]))

    for point in measurement_points:
        point[2] = eg_height + 0.001 - np.tan(angle) * (point[0] - o[0] - roof_width / 2)
        for i in range(0, 3):
            point[i] = int(self.units.convert_length_to_lu(point[i]))
        point[2] += 2

    return inside_mask, measurement_points

# house (2)
proper_single, proper_multi

bei proper_multi mit eg_height

ähnlich zu final/house1, aber anders
* andere Bezeichnung von Länge, Breite des EG
*   house2 hat roof_height, house_1 hat eg_height
*   house2 hat roof_overhang (!) auch in weiterer Berechnung
* house2 berechnet roof_width und eg_height
* andere Benennungen in den logical_ands (eg_x_length vs. eg_length etc.)
* house1 besitzt 1-2 Zeilen mehr in der inside_mask Belegung (andere zeilen sind bei house2 auskommentiert)
* house2 fügt weitere measurement_points hinzu ("auf der Vorderseite vom Dach?") -> anscheinend teilt er auf Vorder- und Rückseite auf (jeweils eg_length/2+overhang) einmal +o und einmal -o (aber je nachdem wo "o" dann ist, ist man damit doch im Negativen?"
*   house2 fügt noch weitere Punkte hinzu in z-Höhe 8 und 18...?
*   house2 fügt noch weiteren Punkte (LU-grid) vor dem Haus hinzu

In [1]:
def house2(self, o, eg_x_length, eg_y_length, roof_height, roof_overhang, roof_width=0, angle=45):
    if roof_width == 0:
        roof_width=eg_y_length
    o[2] = 0
    angle = angle * np.pi / 180
    eg_height = roof_height - (eg_x_length / 2 + roof_overhang) * np.tan(angle)
    self.eg_height = eg_height
    inside_mask = np.zeros_like(self.grid.global_grid()[0], dtype=bool)
    inside_mask = np.where(np.logical_and(
        np.logical_and(np.logical_and(o[0] - eg_x_length / 2 < self.grid.global_grid()[0], self.grid.global_grid()[0] < o[0] + eg_x_length / 2),
                       np.logical_and(o[1] - eg_y_length / 2 < self.grid.global_grid()[1], self.grid.global_grid()[1] < o[1] + eg_y_length / 2)),
                        np.logical_and(o[2] - eg_height <= self.grid.global_grid()[2], self.grid.global_grid()[2] <= o[2] + eg_height)), True, inside_mask)
    inside_mask = np.where(np.logical_and(
                    np.logical_and(np.logical_and(o[0] - eg_x_length / 2 - roof_overhang < self.grid.global_grid()[0], self.grid.global_grid()[0] < o[0] + eg_x_length / 2 + roof_overhang),
                    np.logical_and(o[1] - roof_width / 2 < self.grid.global_grid()[1], self.grid.global_grid()[1] < o[1] + roof_width / 2)),
                    np.logical_and(o[2] + eg_height < self.grid.global_grid()[2],
                    self.grid.global_grid()[2] < o[2] + roof_height + 0.001 - np.tan(angle) * np.abs(self.grid.global_grid()[0] - o[0]))), True, inside_mask)
    #inside_mask = np.where(np.logical_and(
    #                np.logical_and(np.logical_and(o[0] - eg_x_length / 2 - roof_overhang < self.grid.global_grid()[0],self.grid.global_grid()[0] < o[0] + eg_x_length / 2 + roof_overhang),
    #                np.logical_and(o[1] - roof_width / 2 < self.grid.global_grid()[1],self.grid.global_grid()[1] < o[1] + roof_width / 2)),
    #                np.logical_and(o[2] + eg_height < self.grid.global_grid()[2],
    #                self.grid.global_grid()[2] < o[2] + eg_height + self.units.convert_length_to_pu(1.5))), True, inside_mask)

    """make masks for fs to be bounced / not streamed by going over all obstacle points and
    following all e_i's to find neighboring points and which of their fs point towards the obstacle
    (fs pointing to obstacle are added to no_stream_mask, fs pointing away are added to bouncedFs)"""

    x, y, z = inside_mask.shape
    outgoing_mask = np.zeros((self.units.lattice.Q, x, y, z), dtype=bool)
    a, b, c = np.where(inside_mask)
    for p in range(0, len(a)):
        for i in range(0, self.units.lattice.Q):
            try:  # try in case the neighboring cell does not exist (an f pointing out of simulation domain)
                if not inside_mask[a[p] + self.units.lattice.stencil.e[i, 0], b[p] + self.units.lattice.stencil.e[i, 1], c[p] + self.units.lattice.stencil.e[i, 2]]:
                    outgoing_mask[i, a[p] + self.units.lattice.stencil.e[i, 0], b[p] + self.units.lattice.stencil.e[i, 1], c[p] + self.units.lattice.stencil.e[i, 2]] = 1
            except IndexError:
                pass  # just ignore this iteration since there is no neighbor there

    #auf dem Dach:
    measurement_points = []
    measurement_points.append(o + np.asarray([(eg_y_length / 2 + roof_overhang) * 0.25, -roof_width * 0.25, 0]))
    measurement_points.append(o + np.asarray([(eg_y_length / 2 + roof_overhang) * 0.75, -roof_width * 0.25, 0]))
    measurement_points.append(o + np.asarray([(eg_y_length / 2 + roof_overhang) * 0.25, roof_width * 0.25, 0]))
    measurement_points.append(o + np.asarray([(eg_y_length / 2 + roof_overhang) * 0.75, roof_width * 0.25, 0]))
    measurement_points.append(o + np.asarray([(eg_y_length / 2 + roof_overhang) * 0.5, 0, 0]))

    #auf der Vorderseite vom Dach?

    measurement_points.append(o - np.asarray([(eg_y_length / 2 + roof_overhang) * 0.25, -roof_width * 0.25, 0]))
    measurement_points.append(o - np.asarray([(eg_y_length / 2 + roof_overhang) * 0.75, -roof_width * 0.25, 0]))
    measurement_points.append(o - np.asarray([(eg_y_length / 2 + roof_overhang) * 0.25, roof_width * 0.25, 0]))
    measurement_points.append(o - np.asarray([(eg_y_length / 2 + roof_overhang) * 0.75, roof_width * 0.25, 0]))
    measurement_points.append(o - np.asarray([(eg_y_length / 2 + roof_overhang) * 0.5, 0, 0]))

    for point in measurement_points:
        point[2] = o[2] + roof_height + 0.001 - np.tan(angle) * np.abs(point[0] - o[0])
        for i in range(0, 3):
            point[i] = int(self.units.convert_length_to_lu(point[i]))
        point[2] += 2

    measurement_points += [point + np.array([0, 0, 8]) for point in measurement_points[0:10]]
    measurement_points += [point + np.array([0, 0, 18]) for point in measurement_points[0:10]]

        # im freien Raum vor dem Haus

    measurement_points.append(np.asarray([int(self.units.convert_length_to_lu(x)) for x in [self.grid.shape_pu[0] * 0.05, self.grid.shape_pu[1] / 2, self.grid.shape_pu[2] / 2]]))

    return inside_mask, measurement_points

## house 2b (final, final_new2_noverhang)

ggü. house2
* 2b speichert roof_height anstelle eg_height als Attribut
* 2b hat keine outgoing_mask
* measurement_points werden anders definiert...

In [ ]:
def house2(self, o, eg_x_length, eg_y_length, roof_height, roof_overhang, roof_width=0, angle=45):
    if roof_width == 0:
        roof_width=eg_y_length
    o[2] = 0
    angle = angle * np.pi / 180
    eg_height = roof_height - (eg_x_length / 2 + roof_overhang) * np.tan(angle)
    self.roof_height = roof_height
    inside_mask = np.zeros_like(self.grid.global_grid()[0], dtype=bool)
    inside_mask = np.where(np.logical_and(
        np.logical_and(np.logical_and(o[0] - eg_x_length / 2 < self.grid.global_grid()[0], self.grid.global_grid()[0] < o[0] + eg_x_length / 2),
                       np.logical_and(o[1] - eg_y_length / 2 < self.grid.global_grid()[1], self.grid.global_grid()[1] < o[1] + eg_y_length / 2)),
                        np.logical_and(o[2] - eg_height <= self.grid.global_grid()[2], self.grid.global_grid()[2] <= o[2] + eg_height)), True, inside_mask)
    inside_mask = np.where(np.logical_and(
                    np.logical_and(np.logical_and(o[0] - eg_x_length / 2 - roof_overhang < self.grid.global_grid()[0], self.grid.global_grid()[0] < o[0] + eg_x_length / 2 + roof_overhang),
                    np.logical_and(o[1] - roof_width / 2 < self.grid.global_grid()[1], self.grid.global_grid()[1] < o[1] + roof_width / 2)),
                    np.logical_and(o[2] + eg_height < self.grid.global_grid()[2],
                    self.grid.global_grid()[2] < o[2] + roof_height + 0.001 - np.tan(angle) * np.abs(self.grid.global_grid()[0] - o[0]))), True, inside_mask)
    #inside_mask = np.where(np.logical_and(
    #                np.logical_and(np.logical_and(o[0] - eg_x_length / 2 - roof_overhang < self.grid.global_grid()[0],self.grid.global_grid()[0] < o[0] + eg_x_length / 2 + roof_overhang),
    #                np.logical_and(o[1] - roof_width / 2 < self.grid.global_grid()[1],self.grid.global_grid()[1] < o[1] + roof_width / 2)),
    #                np.logical_and(o[2] + eg_height < self.grid.global_grid()[2],
    #                self.grid.global_grid()[2] < o[2] + eg_height + self.units.convert_length_to_pu(1.5))), True, inside_mask)

    #auf dem Dach:
    x = np.linspace(-0.1, 1, 7, endpoint=True)
    y = np.linspace(-0.5, 0.5, 11, endpoint=True)
    xy = np.meshgrid(y, x)
    measurement_points = np.asarray([(eg_x_length / 2 + roof_overhang) * xy[0], roof_width * xy[1], np.zeros_like(xy[0])])

    meas_points = []
    meas_points2 = []
    meas_points3 = []
    meas_points4 = []

    for y in range(0, 10):
        for x in range(0, 6):
            measurement_points[:, x, y] = measurement_points[:, x, y] + o
            measurement_points[2, x, y] = roof_height + 0.001 - np.tan(angle) * np.abs(measurement_points[0, x, y] - o[0])
            measurement_points[:, x, y] = np.round(self.units.convert_length_to_lu(measurement_points[:, x, y])).astype(np.int32)
            meas_points.append(measurement_points[:, x, y])
            meas_points2.append(measurement_points[:, x, y] + np.asarray([0, 0, 1]))
            meas_points3.append(measurement_points[:, x, y] + np.asarray([0, 0, 2]))
            meas_points4.append(measurement_points[:, x, y] + np.asarray([0, 0, 3]))

    measurement_points = meas_points + meas_points2 + meas_points3  + meas_points4

    middle_front = np.asarray([(eg_x_length / 2 + roof_overhang) * -0.25, 0, roof_height + 0.001 - np.tan(angle) * np.abs((eg_x_length / 2 + roof_overhang) * -0.25 - o[0])]) + np.asarray(o)
    for i in range(0, 3):
        measurement_points.append(np.round(self.units.convert_length_to_lu(middle_front + np.asarray([0,0,i]))).astype(np.int32))
    # Hinter dem Einlass
    measurement_points.append(np.asarray([int(self.units.convert_length_to_lu(x)) for x in
                                          [self.grid.shape_pu[0] * 0.05, self.grid.shape_pu[1] / 2,
                                           self.grid.shape_pu[2] / 2]]))

    return inside_mask, measurement_points

# house 3

## house 3a 
proper_single (vgl. newMPI unten)

vs. house2b
- (!) Belegung der inside_mask in house3a OHNE logical_and und where, sondern mit direkter indizierung
- eg_height (h3a) vs. Berechnung aus roof_height (h2b)
- meas_points linspace x bis 0.5 (h3a) vs. bis 1 (h2b)
-  h3a: meas_points-schleife teils per "if 0" deaktiviert, meas_points werden vor Rückgabe leer gesetzt... -> also wohl nicht verwendet


In [ ]:
def house3(self, o, eg_x_length, eg_y_length, eg_height, roof_overhang, roof_width=0, angle=45):
    # In LU, weil da sonst immer alles nicht stimmt^^ alles außer Winkel ist jetzt in LU roof_width gibts nich
    angle = angle * np.pi / 180
    self.eg_height = eg_height
    roof_width = eg_y_length
    inside_mask = np.zeros_like(self.grid.global_grid()[0], dtype=bool)
    inside_mask[int(o[0] - eg_x_length / 2):int(o[0] + eg_x_length / 2),
    int(o[1] - eg_y_length / 2):int(o[1] + eg_y_length / 2), :eg_height] = True
    inside_mask[int(o[0] - eg_x_length / 2 - roof_overhang):int(o[0] + eg_x_length / 2 + roof_overhang),
    int(o[1] - eg_y_length / 2):int(o[1] + eg_y_length / 2), eg_height] = True
    inside_mask[int(o[0] - eg_x_length / 2 - roof_overhang):int(o[0] + eg_x_length / 2 + roof_overhang),
    int(o[1] - eg_y_length / 2):int(o[1] + eg_y_length / 2), eg_height + 1:] = \
        np.where(np.logical_and(self.units.convert_length_to_lu(
            self.grid()[2][int(o[0] - eg_x_length / 2 - roof_overhang):int(o[0] + eg_x_length / 2 + roof_overhang),
            int(o[1] - eg_y_length / 2):int(o[1] + eg_y_length / 2),
            eg_height + 1:]) < o[2] + eg_height + 0.001 - np.tan(angle) * (self.units.convert_length_to_lu(
            self.grid()[0][int(o[0] - eg_x_length / 2 - roof_overhang):int(o[0] + eg_x_length / 2 + roof_overhang),
            int(o[1] - eg_y_length / 2):int(o[1] + eg_y_length / 2),
            eg_height + 1:]) - o[0] - eg_x_length/2),self.units.convert_length_to_lu(self.grid()[2][int(o[0] - eg_x_length / 2 - roof_overhang):int(
                                        o[0] + eg_x_length / 2 + roof_overhang),
                                    int(o[1] - eg_y_length / 2):int(o[1] + eg_y_length / 2),
                                    eg_height + 1:])
                                < o[2] + eg_height + 0.001 + np.tan(angle) * (self.units.convert_length_to_lu(
                                    self.grid()[0][int(o[0] - eg_x_length / 2 - roof_overhang):int(
                                        o[0] + eg_x_length / 2 + roof_overhang),
                                    int(o[1] - eg_y_length / 2):int(o[1] + eg_y_length / 2),
                                    eg_height + 1:]) - o[0] + eg_x_length/2)), True, inside_mask[int(
            o[0] - eg_x_length / 2 - roof_overhang):int(o[0] + eg_x_length / 2 + roof_overhang), int(
            o[1] - eg_y_length / 2):int(o[1] + eg_y_length / 2), eg_height + 1:])
    x = np.linspace(-0.1, 0.5, 7, endpoint=True)
    y = np.linspace(-0.5, 0.5, 11, endpoint=True)
    xy = np.meshgrid(y, x)
    # xy[x/y, x, y]
    measurement_points = np.asarray([(eg_x_length / 2 + roof_overhang) * xy[1], eg_y_length * xy[0], np.zeros_like(xy[0])])

    meas_points = []
    meas_points2 = []
    meas_points3 = []
    meas_points4 = []

    if 0:
        for y in range(0, 10):
            for x in range(0, 6):
                measurement_points[:, x, y] = measurement_points[:, x, y] + o
                measurement_points[2, x, y] = roof_height + 0.001 - np.tan(angle) * np.abs(measurement_points[0, x, y] - o[0])
                measurement_points[:, x, y] = measurement_points[:, x, y].astype(np.int32)
                meas_points.append(measurement_points[:, x, y].astype(np.int32))
                meas_points2.append(measurement_points[:, x, y] + np.asarray([0, 0, 1]))
                meas_points3.append(measurement_points[:, x, y] + np.asarray([0, 0, 2]))
                meas_points4.append(measurement_points[:, x, y] + np.asarray([0, 0, 3]))

        measurement_points = meas_points + meas_points2 + meas_points3  + meas_points4

        middle_front = np.asarray([(eg_x_length / 2 + roof_overhang) * -0.25, 0, roof_height + 0.001 - np.tan(angle) * np.abs((eg_x_length / 2 + roof_overhang) * -0.25)]) + np.asarray(o)
        for i in range(0, 3):
            measurement_points.append(middle_front.astype(np.int32) + np.asarray([0, 0, i]))
        # Hinter dem Einlass
        measurement_points.append(np.asarray([int(self.units.convert_length_to_lu(x)) for x in
                                              [self.grid.shape_pu[0] * 0.05, self.grid.shape_pu[1] / 2,
                                               self.grid.shape_pu[2] / 2]]))
    measurement_points=[]
    return inside_mask, measurement_points

## house 3a, newMPI

vs. house3a
- newMPI: eg_heigth wird berechnet vs. 3a wird übergeben
- newMPI: self.rgrid anstelle self.grid
- inside_mask: roof_height anstelle eg_height (wahrscheinlich Bezugslänge für die charakteristische Länge?)
- inside_mask: es wird auch das berechnete eg_height (oben) verwendet und nicht jeweils aus roof_height mehrfach berechnet (3a)
- daher ist da auch bei 3a ein tangens mehr drin, etc. (vermutlich)
- newMPI: roof_overhang=6 wird überschrieben
- newMPI: measurement_points ist aktiv und parametriert (bis len(y1) anstelle bis 10
- newMPI: middle_front leicht anders definiert (-34/36) anstelle -0.25

newMPI scheint noch neuer zu sein, mehr ist aktiv, roof_height und eg_height werden anders umgerechnet bzw. verwendet vielleicht aber mit dem gleichen Resultat. rgrid anstelle grid wird verwendet, weil man in MPI ist...

In [ ]:
def house3(self, o, eg_x_length, eg_y_length, roof_height, roof_overhang, roof_width=0, angle=45):
    #In LU, weil da sonst immer alles nicht stimmt^^ alles außer Winkel ist jetzt in LU roof_width gibts nich
    angle = angle * np.pi / 180
    eg_height = int(round(roof_height - (eg_x_length / 2 + roof_overhang) * np.tan(angle)))
    self.eg_height = eg_height
    self.roof_height = roof_height
    inside_mask = np.zeros_like(self.rgrid.global_grid()[0], dtype=bool)
    inside_mask[int(o[0]-eg_x_length/2):int(o[0]+eg_x_length/2), int(o[1]-eg_y_length/2):int(o[1]+eg_y_length/2), :eg_height] = True
    inside_mask[int(o[0]-eg_x_length/2-roof_overhang):int(o[0]+eg_x_length/2+roof_overhang), int(o[1]-eg_y_length/2):int(o[1]+eg_y_length/2), eg_height] = True
    inside_mask[int(o[0] - eg_x_length / 2-roof_overhang):int(o[0] + eg_x_length / 2+roof_overhang), int(o[1] - eg_y_length / 2):int(o[1] + eg_y_length / 2), eg_height+1:] = \
        np.where(self.units.convert_length_to_lu(self.rgrid.global_grid()[2][int(o[0] - eg_x_length / 2-roof_overhang):int(o[0] + eg_x_length / 2+roof_overhang), int(o[1] - eg_y_length / 2):int(o[1] + eg_y_length / 2),
    eg_height+1:]) < o[2] + roof_height + 0.5 - np.tan(angle) * np.abs(self.units.convert_length_to_lu(self.rgrid.global_grid()[0][int(o[0] - eg_x_length / 2-roof_overhang):int(o[0] + eg_x_length / 2+roof_overhang), int(o[1] - eg_y_length / 2):int(o[1] + eg_y_length / 2),
    eg_height+1:]) - o[0]), True, inside_mask[int(o[0] - eg_x_length / 2-roof_overhang):int(o[0] + eg_x_length / 2+roof_overhang), int(o[1] - eg_y_length / 2):int(o[1] + eg_y_length / 2), eg_height+1:])

    x1 = np.linspace(-0.1, 1.0, 12, endpoint=True)
    y1 = np.linspace(-0.5, 0.5, 11, endpoint=True)
    xy = np.meshgrid(y1, x1)
    # xy[x/y, x, y]
    roof_overhang = 6
    measurement_points = np.asarray([(eg_x_length / 2 + roof_overhang) * xy[1], eg_y_length * xy[0], np.zeros_like(xy[0])])

    meas_points = []
    meas_points2 = []
    meas_points3 = []
    meas_points4 = []

    for y in range(0, len(y1)):
        for x in range(0, len(x1)):
            measurement_points[:, x, y] = measurement_points[:, x, y] + o
            measurement_points[2, x, y] = roof_height + 0.001 - np.tan(angle) * np.abs(measurement_points[0, x, y] - o[0])
            measurement_points[:, x, y] = measurement_points[:, x, y].astype(np.int32)
            meas_points.append(measurement_points[:, x, y].astype(np.int32))
            meas_points2.append(measurement_points[:, x, y] + np.asarray([0, 0, 1]))
            meas_points3.append(measurement_points[:, x, y] + np.asarray([0, 0, 2]))
            meas_points4.append(measurement_points[:, x, y] + np.asarray([0, 0, 3]))

    measurement_points = meas_points + meas_points2 + meas_points3  + meas_points4

    middle_front = np.asarray([(eg_x_length / 2 + roof_overhang) * (-34/36), 0, roof_height + 0.001 - np.tan(angle) * np.abs((eg_x_length / 2 + roof_overhang) * -0.25)]) + np.asarray(o)
    for i in range(0, 3):
        measurement_points.append(middle_front.astype(np.int32) + np.asarray([0, 0, i]))
    # Hinter dem Einlass
    measurement_points.append(np.asarray([int(self.units.convert_length_to_lu(x)) for x in
                                          [self.rgrid.shape_pu[0] * 0.05, self.rgrid.shape_pu[1] / 2,
                                           self.rgrid.shape_pu[2] / 2]]))
    return inside_mask, measurement_points

## house 3b (final)

vs. newMPI
- grid/rgrid
- x/y vs. x1, y1
- Ende der meas_points for-Schleife mit len(y1) für newMPI und fest 10 bei 3b_final
- middle_front kleine Änderung: (-29/30) final vs. (-34/36) newMPI
- in final steht aber die middle_front Berechnung aus newMPI auch auskommentiert drin als "in house3D_final_new2_noverhang" !

In [ ]:
def house3(self, o, eg_x_length, eg_y_length, roof_height, roof_overhang, roof_width=0, angle=45):
    #In LU, weil da sonst immer alles nicht stimmt^^ alles außer Winkel ist jetzt in LU roof_width gibts nich
    angle = angle * np.pi / 180
    eg_height = int(round(roof_height - (eg_x_length / 2 + roof_overhang) * np.tan(angle)))
    self.eg_height = eg_height
    self.roof_height = roof_height
    inside_mask = np.zeros_like(self.grid.global_grid()[0], dtype=bool)
    inside_mask[int(o[0]-eg_x_length/2):int(o[0]+eg_x_length/2), int(o[1]-eg_y_length/2):int(o[1]+eg_y_length/2), :eg_height] = True
    inside_mask[int(o[0]-eg_x_length/2-roof_overhang):int(o[0]+eg_x_length/2+roof_overhang), int(o[1]-eg_y_length/2):int(o[1]+eg_y_length/2), eg_height] = True
    inside_mask[int(o[0] - eg_x_length / 2-roof_overhang):int(o[0] + eg_x_length / 2+roof_overhang), int(o[1] - eg_y_length / 2):int(o[1] + eg_y_length / 2), eg_height+1:] = \
        np.where(self.units.convert_length_to_lu(self.grid()[2][int(o[0] - eg_x_length / 2-roof_overhang):int(o[0] + eg_x_length / 2+roof_overhang), int(o[1] - eg_y_length / 2):int(o[1] + eg_y_length / 2),
    eg_height+1:]) < o[2] + roof_height + 0.5 - np.tan(angle) * np.abs(self.units.convert_length_to_lu(self.grid()[0][int(o[0] - eg_x_length / 2-roof_overhang):int(o[0] + eg_x_length / 2+roof_overhang), int(o[1] - eg_y_length / 2):int(o[1] + eg_y_length / 2),
    eg_height+1:]) - o[0]), True, inside_mask[int(o[0] - eg_x_length / 2-roof_overhang):int(o[0] + eg_x_length / 2+roof_overhang), int(o[1] - eg_y_length / 2):int(o[1] + eg_y_length / 2), eg_height+1:])

    x = np.linspace(-0.1, 1, 12, endpoint=True)
    y = np.linspace(-0.5, 0.5, 11, endpoint=True)
    xy = np.meshgrid(y, x)
    # xy[x/y, x, y]
### in house3D_final_new2_noverhang: roof_overhang = 6
    measurement_points = np.asarray([(eg_x_length / 2 + roof_overhang) * xy[1], eg_y_length * xy[0], np.zeros_like(xy[0])])

    meas_points = []
    meas_points2 = []
    meas_points3 = []
    meas_points4 = []

    for y in range(0, 10):
        for x in range(0, 6):
            measurement_points[:, x, y] = measurement_points[:, x, y] + o
            measurement_points[2, x, y] = roof_height + 0.001 - np.tan(angle) * np.abs(measurement_points[0, x, y] - o[0])
            measurement_points[:, x, y] = measurement_points[:, x, y].astype(np.int32)
            meas_points.append(measurement_points[:, x, y].astype(np.int32))
            meas_points2.append(measurement_points[:, x, y] + np.asarray([0, 0, 1]))
            meas_points3.append(measurement_points[:, x, y] + np.asarray([0, 0, 2]))
            meas_points4.append(measurement_points[:, x, y] + np.asarray([0, 0, 3]))

    measurement_points = meas_points + meas_points2 + meas_points3  + meas_points4

    middle_front = np.asarray([(eg_x_length / 2 + roof_overhang) * (-29/30), 0, roof_height + 0.001 - np.tan(angle) * np.abs((eg_x_length / 2 + roof_overhang) * -0.25)]) + np.asarray(o)
### in house3D_final_new2_noverhang: middle_front = np.asarray([(eg_x_length / 2 + roof_overhang) * (-34/36), 0, roof_height + 0.001 - np.tan(angle) * np.abs((eg_x_length / 2 + roof_overhang) * -0.25)]) + np.asarray(o)    
    for i in range(0, 3):
        measurement_points.append(middle_front.astype(np.int32) + np.asarray([0, 0, i]))
    # Hinter dem Einlass
    measurement_points.append(np.asarray([int(self.units.convert_length_to_lu(x)) for x in
                                          [self.grid.shape_pu[0] * 0.05, self.grid.shape_pu[1] / 2,
                                           self.grid.shape_pu[2] / 2]]))
    return inside_mask, measurement_points

## house 3c (3D_literature, 3D_literature_neueBoundary)

vs. house3a
- rgrid (literature) vs. grid (house3a)
- inside_mask quasi gleich
- house3c_lit hat wesentlich unkompliziertere meas_points... diese sind nur eine Schleife über range(0,eg_x_length+1)

In [ ]:
def house3(self, o, eg_x_length, eg_y_length, eg_height, roof_overhang, roof_width=0, angle=45):
    # In LU, weil da sonst immer alles nicht stimmt^^ alles außer Winkel ist jetzt in LU roof_width gibts nich
    angle = angle * np.pi / 180
    self.eg_height = int(eg_height)
    eg_height = self.eg_height
    roof_width = eg_y_length
    inside_mask = np.zeros_like(self.rgrid.global_grid()[0], dtype=bool)
    inside_mask[int(o[0] - eg_x_length / 2):int(o[0] + eg_x_length / 2),
    int(o[1] - eg_y_length / 2):int(o[1] + eg_y_length / 2), :eg_height] = True
    inside_mask[int(o[0] - eg_x_length / 2 - roof_overhang):int(o[0] + eg_x_length / 2 + roof_overhang),
    int(o[1] - eg_y_length / 2):int(o[1] + eg_y_length / 2), eg_height] = True
    inside_mask[int(o[0] - eg_x_length / 2 - roof_overhang):int(o[0] + eg_x_length / 2 + roof_overhang),
    int(o[1] - eg_y_length / 2):int(o[1] + eg_y_length / 2), eg_height + 1:] = \
        np.where(np.logical_and(self.units.convert_length_to_lu(
            self.grid[2][int(o[0] - eg_x_length / 2 - roof_overhang):int(o[0] + eg_x_length / 2 + roof_overhang),
            int(o[1] - eg_y_length / 2):int(o[1] + eg_y_length / 2),
            eg_height + 1:]) < o[2] + eg_height + 0.001 - np.tan(angle) * (self.units.convert_length_to_lu(
            self.grid[0][int(o[0] - eg_x_length / 2 - roof_overhang):int(o[0] + eg_x_length / 2 + roof_overhang),
            int(o[1] - eg_y_length / 2):int(o[1] + eg_y_length / 2),
            eg_height + 1:]) - o[0] - eg_x_length / 2), self.units.convert_length_to_lu(
            self.grid[2][int(o[0] - eg_x_length / 2 - roof_overhang):int(
                o[0] + eg_x_length / 2 + roof_overhang),
            int(o[1] - eg_y_length / 2):int(o[1] + eg_y_length / 2),
            eg_height + 1:])< o[2] + eg_height + 0.001 + np.tan(angle) * (self.units.convert_length_to_lu(
            self.grid[0][int(o[0] - eg_x_length / 2 - roof_overhang):int(
                o[0] + eg_x_length / 2 + roof_overhang),
            int(o[1] - eg_y_length / 2):int(o[1] + eg_y_length / 2),
            eg_height + 1:]) - o[0] + eg_x_length / 2)), True, inside_mask[int(
            o[0] - eg_x_length / 2 - roof_overhang):int(o[0] + eg_x_length / 2 + roof_overhang), int(
            o[1] - eg_y_length / 2):int(o[1] + eg_y_length / 2), eg_height + 1:])

    measurement_points = []
    for x in range(0, eg_x_length+1):
        _x = o[0] - int(eg_x_length / 2) + x
        measurement_points.append([_x, o[1], eg_height + np.tan(angle) * (eg_x_length/2 - np.abs(o[0] - _x)) + 3])

    return inside_mask, measurement_points

## house (4) (TestGeometry)
# (!) braucht das "Geometry3D Package!

In [ ]:
def house4(self, o, eg_x_length, eg_y_length, roof_height, roof_overhang, roof_width=0, angle=45):
    angle = angle * np.pi / 180
    eg_height = int(round(roof_height - (eg_x_length / 2 + roof_overhang) * np.tan(angle)))

    ridge_0 = gd.Point(eg_x_length / 2 + roof_overhang, 0, roof_height)
    ridge_1 = gd.Point(eg_x_length / 2 + roof_overhang, eg_y_length, roof_height)

    roof_corner_0 = gd.Point(0, 0, eg_height)
    roof_corner_1 = gd.Point(0, eg_y_length, eg_height)
    roof_corner_2 = gd.Point(eg_x_length + 2 * roof_overhang, eg_y_length, eg_height)
    roof_corner_3 = gd.Point(eg_x_length + 2 * roof_overhang, 0, eg_height)

    eg_corner_0_up = gd.Point(roof_overhang, 0, eg_height)
    eg_corner_1_up = gd.Point(roof_overhang, eg_y_length, eg_height)
    eg_corner_2_up = gd.Point(eg_x_length + roof_overhang, eg_y_length, eg_height)
    eg_corner_3_up = gd.Point(eg_x_length + roof_overhang, 0, eg_height)

    eg_corner_0_down = gd.Point(roof_overhang, 0, 0)
    eg_corner_1_down = gd.Point(roof_overhang, eg_y_length, 0)
    eg_corner_2_down = gd.Point(eg_x_length + roof_overhang, eg_y_length, 0)
    eg_corner_3_down = gd.Point(eg_x_length + roof_overhang, 0, 0)

    r = gd.Renderer()
    r.add((ridge_0, 'r', 1))
    r.add((ridge_1, 'r', 1))

    r.add((roof_corner_0, 'r', 1))
    r.add((roof_corner_1, 'r', 1))
    r.add((roof_corner_2, 'r', 1))
    r.add((roof_corner_3, 'r', 1))

    r.add((eg_corner_0_up, 'r', 1))
    r.add((eg_corner_1_up, 'r', 1))
    r.add((eg_corner_2_up, 'r', 1))
    r.add((eg_corner_3_up, 'r', 1))

    r.add((eg_corner_0_down, 'r', 1))
    r.add((eg_corner_1_down, 'r', 1))
    r.add((eg_corner_2_down, 'r', 1))
    r.add((eg_corner_3_down, 'r', 1))

    wxn = gd.ConvexPolygon((eg_corner_0_up, eg_corner_0_down, eg_corner_1_down, eg_corner_1_up))
    wyp = gd.ConvexPolygon((eg_corner_2_up, eg_corner_2_down, eg_corner_1_down, eg_corner_1_up))
    wxp = gd.ConvexPolygon((eg_corner_2_up, eg_corner_2_down, eg_corner_3_down, eg_corner_3_up))
    wyn = gd.ConvexPolygon((eg_corner_0_up, eg_corner_0_down, eg_corner_3_down, eg_corner_3_up))
    boden = gd.ConvexPolygon((eg_corner_0_down, eg_corner_1_down, eg_corner_2_down, eg_corner_3_down))
    decke = gd.ConvexPolygon((eg_corner_0_up, eg_corner_1_up, eg_corner_2_up, eg_corner_3_up))

    eg = gd.ConvexPolyhedron((wxn, wyp, wxp, wyn, boden, decke))

    boden = gd.ConvexPolygon((roof_corner_0, roof_corner_1, roof_corner_2, roof_corner_3))
    dfn = gd.ConvexPolygon((roof_corner_0, roof_corner_1, ridge_0, ridge_1))
    dfp = gd.ConvexPolygon((roof_corner_2, roof_corner_3, ridge_0, ridge_1))
    sp = gd.ConvexPolygon((roof_corner_0, roof_corner_3, ridge_0))
    sn = gd.ConvexPolygon((roof_corner_1, roof_corner_2, ridge_1))

    dach = gd.ConvexPolyhedron((boden, dfp, dfn, sp, sn))
    r.add((eg, 'g', 1))
    r.add((dach, 'g', 1))
    r.show()


    o = gd.Vector(o)
    dach.move((o))
    eg.move((o))

    inside_mask = np.zeros_like(self.rgrid.global_grid()[0], dtype=bool)

    for x in range(0, self.rgrid.global_shape[0]):
        print(x)
        for y in range(0, self.rgrid.global_shape[1]):
            for z in range(0, self.rgrid.global_shape[2]):
                point = gd.Point((x, y, z))
                if dach.__contains__(point) or eg.__contains__(point):
                    inside_mask[x,y,z] = True
    print("blub")